In [1]:
import numpy as np

from exputils.dot.dot_product import compute_all_dot_products
from exputils.RoM.dot import calculate_RoM_dot
from exputils.st_norm import lb_by_st_norm
from exputils.state.canonical_magic_state import (
    make_canonical_magic_state_in_pauli_basis,
)
from exputils.state.random import make_random_quantum_state

In [13]:
def lb_by_dot(n_qubit, rho_vec):
    dots = compute_all_dot_products(n_qubit, rho_vec)
    dp = np.max(dots)
    dm = np.min(dots)
    assert np.isclose(rho_vec.T @ rho_vec, np.linalg.norm(rho_vec, ord=2) ** 2)
    return 1 + 2 * max(0, (np.linalg.norm(rho_vec, ord=2) ** 2 - dp) / (dp - dm))


def lb_by_stabilizer_fidelity(n_qubit, rho_vec):
    # valid only when the state is pure
    dots = compute_all_dot_products(n_qubit, rho_vec)
    stabilizer_fidelity = np.max(dots) / (2**n_qubit)
    return (rho_vec.T @ rho_vec) / (2**n_qubit) / stabilizer_fidelity

In [14]:
n_qubit = 4

for kind in ["pure", "mixed"]:
    best_lb_count = dict()
    best_lb_count["sf"] = 0
    best_lb_count["st"] = 0
    best_lb_count["dot"] = 0
    for seed in range(100):
        rho_vec = make_random_quantum_state(kind, n_qubit, seed=seed)
        RoM, coeff, Amat = calculate_RoM_dot(n_qubit, rho_vec, method="gurobi")
        # print("RoM = ", RoM)

        p = (RoM - 1) / 2
        rho_p = np.sum(coeff[coeff > 0] * Amat[:, coeff > 0].toarray(), axis=1)
        rho_m = np.sum(coeff[coeff < 0] * Amat[:, coeff < 0].toarray(), axis=1)
        assert np.allclose(rho_p + rho_m, rho_vec)
        assert np.isclose(rho_p[0], 1 + p)
        assert np.isclose(rho_m[0], -p)
        rho_p /= 1 + p
        rho_m /= -p
        assert np.allclose((1 + p) * rho_p + (-p) * rho_m, rho_vec)
        assert np.isclose(rho_p[0], 1)
        assert np.isclose(rho_m[0], 1)

        # print(f"{p=}")
        # print(f"{np.dot(rho_vec, rho_p)=}")
        # print(f"{np.dot(rho_vec, rho_m)=}")

        dp = np.dot(rho_vec, rho_p)
        dm = np.dot(rho_vec, rho_m)
        # print(f"{np.linalg.norm(rho_vec, ord=2) ** 2=}")
        assert np.isclose(p, (np.linalg.norm(rho_vec, ord=2) ** 2 - dp) / (dp - dm))

        lb_sf = lb_by_stabilizer_fidelity(n_qubit, rho_vec)
        lb_st = lb_by_st_norm(n_qubit, rho_vec)
        lb_dot = lb_by_dot(n_qubit, rho_vec)

        best_lb = (0.0, "None")

        print(f"{lb_sf=}")
        assert lb_sf <= RoM
        if lb_sf > best_lb[0]:
            best_lb = (lb_sf, "sf")

        # print(f"{lb_st=}")
        print(f"{lb_dot=}")
        assert lb_st <= RoM
        assert lb_dot <= RoM

        if lb_st > best_lb[0]:
            best_lb = (lb_st, "st")
        if lb_dot > best_lb[0]:
            best_lb = (lb_dot, "dot")

        best_lb_count[best_lb[1]] += 1

    print(f"{best_lb_count=}")

lb_sf=1.7848266417977787
lb_dot=2.569657374921798
lb_sf=2.08796162723462
lb_dot=3.1759245565249876
lb_sf=1.9674600518646173
lb_dot=2.9349272623431193
lb_sf=1.6791792186872823
lb_dot=2.3583595598779716
lb_sf=1.6942965885644283
lb_dot=2.388595043312849
lb_sf=1.9150265864033176
lb_dot=2.8300637956780155
lb_sf=2.135936687642714
lb_dot=3.2718786047297663
lb_sf=1.7228550010730312
lb_dot=2.445711429770231
lb_sf=1.8867955901216764
lb_dot=2.773593577232412
lb_sf=2.060172114354628
lb_dot=3.120355090833797
lb_sf=2.0333549243360434
lb_dot=3.0667122636818833
lb_sf=2.0916832393016875
lb_dot=3.1833734530904687
lb_sf=2.281815928762279
lb_dot=3.5636376453388627
lb_sf=2.0788567973664582
lb_dot=3.1577323381835973
lb_sf=1.9684029043039881
lb_dot=2.9368181998678278
lb_sf=1.8311579989390963
lb_dot=2.662327136672008
lb_sf=1.7794862789173738
lb_dot=2.558974192985572
lb_sf=2.0817902791754554
lb_dot=3.1635937950736914
lb_sf=2.2845795573570227
lb_dot=3.5691910877007254
lb_sf=1.9774830843826663
lb_dot=2.954968980

check the $\langle\langle \rho|\rho \rangle\rangle \leq \langle\langle\rho|\rho_{\text{max}} \rangle\rangle$


In [9]:
n_qubit = 1
rho_vec = make_canonical_magic_state_in_pauli_basis(n_qubit).toarray().flatten()
print(f"{rho_vec=}")

RoM, coeff, Amat = calculate_RoM_dot(n_qubit, rho_vec, method="gurobi")
assert RoM == np.linalg.norm(coeff, ord=1)
print(f"{RoM=}")
print(f"{Amat.toarray()=}")
print(f"{coeff=}")

print(f"{rho_vec.T @ rho_vec=}")
print(f"{max(compute_all_dot_products(n_qubit, rho_vec))=}")

for i, col in enumerate(Amat.toarray().T):
    assert 0 <= col @ rho_vec <= (2**n_qubit)
    print(f"{col @ rho_vec * coeff[i]=}")

rho_vec=array([1.        , 0.70710678, 0.70710678, 0.        ])
RoM=1.414213562373095
Amat.toarray()=array([[ 1,  1,  1],
       [ 1, -1,  0],
       [ 0,  0,  1],
       [ 0,  0,  0]], dtype=int8)
coeff=array([ 0.5       , -0.20710678,  0.70710678])
rho_vec.T @ rho_vec=2.0
max(compute_all_dot_products(n_qubit, rho_vec))=1.7071067811865475
col @ rho_vec * coeff[i]=0.8535533905932737
col @ rho_vec * coeff[i]=-0.06066017177982128
col @ rho_vec * coeff[i]=1.2071067811865475


We check with H state


In [10]:
n_qubit = 6
rho_vec = make_canonical_magic_state_in_pauli_basis(n_qubit).toarray().flatten()

print(f"{lb_by_stabilizer_fidelity(n_qubit, rho_vec)=}")
print(f"{lb_by_st_norm(n_qubit, rho_vec)=}")
print(f"{lb_by_dot(n_qubit, rho_vec)=}")

lb_by_stabilizer_fidelity(n_qubit, rho_vec)=2.5859245482734514
lb_by_st_norm(n_qubit, rho_vec)=3.1269039581923277
lb_by_dot(n_qubit, rho_vec)=4.171849096546902


```python
lb_by_st_norm(n_qubit, rho_vec)=3.1269039581923277
lb_by_dot(n_qubit, rho_vec)=4.171849096546902
true_RoM=4.73894
```

We get the better lower bound by using the dot product.
